In [8]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing as pp
from keras.utils import np_utils
ss = pp.StandardScaler()
import h5py

X_train = np.load("20170827_mawilab_flow_000flags_nosvc_features.pkl")
Y_train = np.load("20170827_mawilab_flow_000flags_nosvc_labels.pkl")


# Test Set 3 : 20170827_mawilab_flow_003
X_test3 = np.load("20170827_mawilab_flow_003flags_nosvc_features.pkl")
Y_test3 = np.load("20170827_mawilab_flow_003flags_nosvc_labels.pkl")
X_train = ss.fit_transform(X_train)
X_test3 = ss.fit_transform(X_test3)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test3 = np.reshape(X_test3, (X_test3.shape[0],X_test3.shape[1],1))


In [9]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(15, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 15, 64)            256       
_________________________________________________________________
activation_7 (Activation)    (None, 15, 64)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 448)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               57472     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [10]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test3, Y_test3) ,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_03.png')

Train on 38361 samples, validate on 42398 samples
Epoch 1/10
38361/38361 [==============================] - 10s 252us/step - loss: 0.4808 - acc: 0.7042 - val_loss: 0.4954 - val_acc: 0.6498
Epoch 2/10
38361/38361 [==============================] - 9s 228us/step - loss: 0.4632 - acc: 0.7179 - val_loss: 0.4873 - val_acc: 0.6453
Epoch 3/10
38361/38361 [==============================] - 9s 235us/step - loss: 0.4598 - acc: 0.7189 - val_loss: 0.4837 - val_acc: 0.6563
Epoch 4/10
38361/38361 [==============================] - 9s 230us/step - loss: 0.4565 - acc: 0.7243 - val_loss: 0.4862 - val_acc: 0.6528
Epoch 5/10
38361/38361 [==============================] - 9s 233us/step - loss: 0.4539 - acc: 0.7265 - val_loss: 0.4859 - val_acc: 0.6586
Epoch 6/10
38361/38361 [==============================] - 9s 232us/step - loss: 0.4509 - acc: 0.7289 - val_loss: 0.4782 - val_acc: 0.6597
Epoch 7/10
38361/38361 [==============================] - 9s 234us/step - loss: 0.4503 - acc: 0.7290 - val_loss: 0.4798 -

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
y_pred = cnn1d_1.predict(X_test3)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = cnn1d_1.predict_classes(X_test3)
print(y_pred)

p=cnn1d_1.predict_proba(X_test3)

target_names = ['normal', 'anomaly']
print(classification_report(np.argmax(Y_test3, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(Y_test3, axis=1), y_pred))
Y_test3 = np.argmax(Y_test3, axis=1)

print(f1_score(Y_test3, y_pred, average='macro'))

print(f1_score(Y_test3, y_pred, average='micro'))

print(f1_score(Y_test3, y_pred, average='weighted'))

print(f1_score(Y_test3, y_pred, average=None))

[[0.502954   0.49703035]
 [0.50108093 0.49893492]
 [0.502954   0.49703035]
 ...
 [0.0034403  0.99661785]
 [0.0034403  0.99661785]
 [0.44749984 0.5537377 ]]
[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]
             precision    recall  f1-score   support

     normal     0.8169    0.3535    0.4935     16258
    anomaly     0.7028    0.9507    0.8082     26140

avg / total     0.7466    0.7217    0.6875     42398

[[ 5748 10510]
 [ 1288 24852]]
0.6508432420331137
0.7217321571772254
0.6875122254635005
[0.49351764 0.80816884]
